# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуем данные и ответим на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [28]:
#создаем новую таблицу
#сортируем по столбцу 'children', сравнивая с 'debt'
#через agg считаем кол-во кредитов, кол-во должников и их соотношение
#Сортируем по 'mean' вместе с reset_index() для более прятного и читаемого вида
#Переводим значения столбца 'mean' в проценты
debt_children_ratio = data.groupby('children')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean').reset_index() 
debt_children_ratio['mean'] = [f'{x:,.2%}' for x in debt_children_ratio['mean']]
debt_children_ratio

,count,sum,mean
,debt,debt,debt
children,,,
4,41,4,9.76%
2,2052,194,9.45%
1,4808,444,9.23%
3,330,27,8.18%
0,14091,1063,7.54%
5,9,0,0.00%


**Вывод:** 

1. У заёмщиков с 5-ю детьми наблюдается безупречная кредитная история, но данные можно считать аномальными, из-за слишком маленькой выборки.
  * Выборка по клиентам с 3-мя и 4-мя детьми тоже относительно мала.
2. В остальном, относительно полученной информации, можно сделать вывод, что тенденции пропорционального роста количества должников относительно роста количества детей заёмщиков не наблюдается.



Если не считать строки с маленькой выборкой, то можно сделать вывод, что процент должников увеличивается пропорционально  количеству детей имеющихся у заёмщиков.
  * Среди клиентов без детей - меньше всего должников
  * Среди клиентов с 2-мя детьми - больше всего
 
 

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
#то же самое, что и в предыдущем задании только меняем 'children' на 'family_status'
debt_family_ratio = data.groupby('family_status')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean').reset_index() 
debt_family_ratio['mean'] = [f'{x:,.2%}' for x in debt_family_ratio['mean']]
debt_family_ratio

,family_status,count,sum,mean
0,вдовец / вдова,951,63,6.62%
1,в разводе,1189,84,7.06%
2,женат / замужем,12261,927,7.56%
3,гражданский брак,4134,385,9.31%
4,Не женат / не замужем,2796,273,9.76%


**Вывод:** 

1. Клиенты-вдовы и в разводе имеют наименьший процент должников.
2. Холостые заёмщики имеют наибольший процент должников.

Исходя из доступной информации можно сделать вывод о том, что холостые клиенты и заёмщики состоящие в гражданском браке - самые финансово безответственные, а клиенты, вышедшие из брака - наиболее.

Также клиенты, состоящие в браке, берут больше всего кредитов.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
#то же самое, что и в предыдущем задании только меняем 'family_status' на 'total_income_category'
debt_income_ratio = data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean').reset_index() 
debt_income_ratio['mean'] = [f'{x:,.2%}' for x in debt_income_ratio['mean']]
debt_income_ratio

,total_income_category,count,sum,mean
0,D,349,21,6.02%
1,B,5014,354,7.06%
2,A,25,2,8.00%
3,C,15921,1353,8.50%
4,E,22,2,9.09%


**Вывод:** 
  * Здесь не наблюдается пропорциональной прогрессии роста процента должников относительно их дохода.
  * Категории клиентов с самыми низкими показателями дохода находятся одновременно в начале и конца списка отсортированного по проценту должников
  * Сложно дать оценку выборки по клиентам с самым низким и клиентам с самым высоким доходом из-за малого количества займов.
  * Заёмщики со средним доходом являются самой распространнёной категорией, ~~и одновреммено самой ненадёжной~~
  -------------
  
Если брать только качественную выборку, то можно сделать вывод, что:
  * Клиенты со средним доходом [C] являются самыми безответственными заёмщиками
  * А клиенты с доходом выше среднего [B] - самыми надежными




#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
##то же самое, что и в предыдущем задании только меняем 'total_income_category' на 'purpose_category'
debt_purpose_ratio = data.groupby('purpose_category')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean').reset_index() 
debt_purpose_ratio['mean'] = [f'{x:,.2%}' for x in debt_purpose_ratio['mean']]
debt_purpose_ratio

,purpose_category,count,sum,mean
0,операции с недвижимостью,10751,780,7.26%
1,проведение свадьбы,2313,183,7.91%
2,получение образования,3988,369,9.25%
3,операции с автомобилем,4279,400,9.35%


**Вывод:** 
1. Люди берущие кредит для образования или операций с автомобилем - больше всех затрудняются вернуть кредит.
2. В то время как клиенты с кредитом на недвижимость и свадьбу - более уверенно его возвращают.


#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

1. Человеческий фактор:
  * Ошибка ввода
  * Возможно клиент пропустил ячейку или забыл ввести данные
  * А может быть и просто не захотел (информация о доходе - довольно чувствительные данные)
  * Пропуски в столбцах о доходе и стаже работы могут быть пропущены просто за неимением этой информации у самого клиента (безработный клиент)
2. Возможны ошибки выгрузки данных со стороны программного обеспечения или человека занимающегося выгрузкой данных.
  

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. В таких случаях медиана определит наиболее логичное значение.

### Шаг 4: общий вывод.

В целом, по исследованным данным можно сделать следующий вывод:

  * Клиенты с **официально оформленными отношениями** (или которые в прошлом были в официальном в браке) и **не имеющие детей**, а также имеющие **доход выше среднего показателя** (200001–1000000) - `самые ответственные заемщики`;
  * Клиенты, состоящие в **неофициальном браке или находящиеся без отношений**, при этом **имеющие 1 или 2 детей**, а также имеющие **средний доход** (50001–200000) - `самые менее ответственные заемщики`.